## **0. Setting**

In [ ]:
# drive.mount
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# import package
import pandas as pd
import json
import re

## **1. 구 별로 나누어져 있는 음식점 데이터 통합**

In [ ]:
# import data (지역구 별 음식점 데이터)
df = pd.read_csv('dir/' sep=',', encoding='CP949')
df2 = pd.read_csv('dir/', sep=',', encoding='CP949')
df3 = pd.read_csv('dir/', sep=',', encoding='CP949')
df4 = pd.read_csv('dir', sep=',', encoding='CP949')

In [ ]:
# 남구 음식점
df.head(3)

,지정년도,업소명,소재지(도로명),소재지(지번),소재지전화번호,업태명,주된음식
0,2018,(유)아웃백스테이크하우스코리아삼산점,"울산광역시 남구 삼산로 231, 울산 센트럴 자이 301~304호 (달동)",울산광역시 남구 달동 1364-1 울산 센트럴 자이,052-256-1491,패밀리레스트랑,스테이크
1,2013,(주)트레비브로이,울산광역시 남구 삼산중로 46 (삼산동),울산광역시 남구 삼산동 1475-1,052-258-0101,경양식,"피자, 스테이크"
2,2005,가무댕댕,울산광역시 남구 대학로94번길 11 (무거동),울산광역시 남구 무거동 630-18,052-224-3331,경양식,돈까스


In [ ]:
# 동구 음식점
df2.head(3)

,연번,인허가일자,업소명,소재지(도로명),소재지(지번),영업장면적,기준일자
0,1,1974-11-27,929막창,울산광역시 동구 방어진순환도로 929 (서부동),울산광역시 동구 서부동 181-31,16,2021-05-06
1,2,1976-11-15,한화장반점,울산광역시 동구 동진4길 52 (방어동),울산광역시 동구 방어동 127-12,31.61,2021-05-06
2,3,1976-07-22,장동희의수제돈까스,"울산광역시 동구 내진길 45, 1층 (방어동)",울산광역시 동구 방어동 198-12,48.77,2021-05-06


In [ ]:
# 울주군 음식점
df3.head(3)

,업종명,업소명,소재지
0,일반음식점,덕하식당,울산광역시 울주군 청량읍 덕하로 263
1,일반음식점,언양국밥,울산광역시 울주군 온양읍 남창장터길 28
2,일반음식점,금화불고기,울산광역시 울주군 언양읍 성리길 1


In [ ]:
# 중구 음식점
df4.head(3)

,가맹점명,소재지도로명주소,소재지지번주소,위도,경도,전화번호,데이터기준일자
0,가마솥 뚝배기,울산광역시 중구 곽남길 20 (남외동),울산광역시 중구 남외동 432-21,35.568959,129.341822,052-297-3177,2021-09-16
1,고건오의티라미슈,울산광역시 중구 신기길 128,울산광역시 중구 태화동 117-2,35.551840,129.300660,052-245-0101,2021-09-16
2,고봉민김밥성남점,울산광역시 중구 학성로 50-1,울산광역시 중구 성남동 190-106,35.553672,129.317168,052-243-3355,2021-09-16


In [ ]:
# data concat
df = df[['업소명', '소재지(도로명)']]
df2 = df2[['업소명', '소재지(도로명)']]
df3 = df3[['업소명', '소재지']]
df4 = df4[['가맹점명', '소재지도로명주소']]
df3 = df3.rename(columns = {'소재지':'소재지(도로명)'})
df4 = df4.rename(columns = {'가맹점명':'업소명', '소재지도로명주소':'소재지(도로명)'})

df_fin = pd.concat([df, df2, df3, df4])
df_fin.reset_index(inplace = True, drop = True)

# url 데이터를 입력할 column 생성
df_fin['naver_map_url'] = ''

# 크롤링을 위한 검색어 데이터 생성
df_fin['검색어'] = df_fin['소재지(도로명)'] + ' ' + df_fin['업소명']

# 결과 확인
df_fin

,업소명,소재지(도로명),naver_map_url,검색어
0,(유)아웃백스테이크하우스코리아삼산점,"울산광역시 남구 삼산로 231, 울산 센트럴 자이 301~304호 (달동)",,"울산광역시 남구 삼산로 231, 울산 센트럴 자이 301~304호 (달동) (유)아..."
1,(주)트레비브로이,울산광역시 남구 삼산중로 46 (삼산동),,울산광역시 남구 삼산중로 46 (삼산동) (주)트레비브로이
2,가무댕댕,울산광역시 남구 대학로94번길 11 (무거동),,울산광역시 남구 대학로94번길 11 (무거동) 가무댕댕
3,가연장미역국정찬,"울산광역시 남구 왕생로 43, 1층 (달동)",,"울산광역시 남구 왕생로 43, 1층 (달동) 가연장미역국정찬"
4,고반식당 울산삼산점,"울산광역시 남구 달삼로 41, 1층 (달동)",,"울산광역시 남구 달삼로 41, 1층 (달동) 고반식당 울산삼산점"
...,...,...,...,...
8172,황궁쟁반짜장,울산광역시 중구 동헌길 26 (우정동),,울산광역시 중구 동헌길 26 (우정동) 황궁쟁반짜장
8173,황궁쟁반짜장(옥뽕),울산광역시 중구 구루미7길 43,,울산광역시 중구 구루미7길 43 황궁쟁반짜장(옥뽕)
8174,황궁쟁반짜장약사도성반점,울산광역시 중구 평산4길 15,,울산광역시 중구 평산4길 15 황궁쟁반짜장약사도성반점
8175,흥부네,울산광역시 중구 서원1길 37(반구동),,울산광역시 중구 서원1길 37(반구동) 흥부네


In [ ]:
# export data
df_fin.to_csv('dir/', header = True, index = True)

## **2. 소재지, 리뷰, 음식점 데이터 병합**

In [ ]:
# import data (리뷰 관련 데이터)
df_1 = pd.read_csv('dir/')
df_2 = pd.read_csv('dir/')
df_3 = pd.read_csv('dir/')
df_4 = pd.read_csv('dir/')

# data concat
df = pd.concat([df_1, df_2, df_3, df_4])
df['review'] = ''

# 결과 확인
df.head()

,Unnamed: 0,UserID,ItemID,Rating,Timestamp,review
0,0,5ead1b1a8f87a842bc999ecf,가무댕댕,3.5,2021.10.13,
1,1,6124c652fde80130e3a864cb,가무댕댕,4.0,2021.10.09,
2,2,5d4e88b1756a76db2d53f502,가무댕댕,4.5,2021.10.01,
3,3,6050bd1c7a3ec9fc3bd58a24,가무댕댕,5.0,2021.09.29,
4,4,5bfe9d3027514b9092aa27c6,가무댕댕,5.0,2021.09.25,


In [ ]:
# import data (리뷰)
with open(url + 'review_1.json', 'r') as f:
    json_data = json.load(f)
with open(url + 'review_2.json', 'r') as f:
    json_data_2 = json.load(f)
with open(url + 'review_3.json', 'r') as f:
    json_data_3 = json.load(f)
with open(url + 'review_4.json', 'r') as f:
    json_data_4 = json.load(f)

# data concat
json_data.update(json_data_2)
json_data.update(json_data_3)
json_data.update(json_data_4)

'''
데이터 형태:
dict = {'매장명' : {'이용자 ID값' : '리뷰 내용'}}
'''

In [ ]:
# 이용자 ID가 매장코드_이용자 ID로 구성되어 있음. 때문에 이를 분리한 후 
# 이용자 ID와 매장명이 데이터 프레임의 값과 일치하는 row에 리뷰 데이터를 추가
li = list(set(df['ItemID']))

for i in li :
  for j, k in json_data[str(i)].items() :
    custo = str(j.split('_')[1])
    # 이용자 ID로만 구분지으면 여러 매장에 댓글을 단 케이스가 있어 리뷰가 잘못 붙을 수 있기 때문에 이에 대한 예외처리 추가
    df.loc[(df['UserID'] == custo) & (df['ItemID'] == str(i)), 'review'] = k

# 결과 확인
df = df[['UserID', 'ItemID', 'Rating', 'Timestamp', 'review']]
df.head(20)

,UserID,ItemID,Rating,Timestamp,review
0,5ead1b1a8f87a842bc999ecf,가무댕댕,3.5,2021.10.13,추억의 가무댕댕\n밥 양 좀 늘려주셨으면 좋겠어요. 서빙하는 남자분도 좀 불친절
1,6124c652fde80130e3a864cb,가무댕댕,4.0,2021.10.09,"물려서 다 못먹은 돈까스 ,, 예전에 비해 맛이 많이 달라졌다"
2,5d4e88b1756a76db2d53f502,가무댕댕,4.5,2021.10.01,맛잇어요
3,6050bd1c7a3ec9fc3bd58a24,가무댕댕,5.0,2021.09.29,맛있게 잘 먹었어요
4,5bfe9d3027514b9092aa27c6,가무댕댕,5.0,2021.09.25,맛있게 잘 먹었습니다~♡
5,5efde3edb4dd01caff09a7d3,가무댕댕,5.0,2021.09.25,돈까스 먹방
6,5d9007078f87a842bce2535d,가무댕댕,5.0,2021.09.25,좋아요
7,6050bd1c7a3ec9fc3bd58a24,가무댕댕,5.0,2021.09.25,맛있게 잘 먹었어요
8,5fd9506eafaeea6203f4217b,가무댕댕,3.0,2021.09.11,울산대학맞으편 오랜역사
9,5f42375da602e98f561556b0,가무댕댕,4.0,2021.09.10,맛있어요!!


In [ ]:
# 결과 확인
df.shape

(15483, 6)

In [ ]:
# 소재지 데이터와 리뷰 데이터 병합
df_info = pd.read_csv('dir/')
df_info = df_info[['업소명', '소재지(도로명)']]

df_fin = pd.merge(df, df_info, how = 'inner', left_on = 'ItemID', right_on = '업소명')

# 필요한 칼럼 추출 및 결측치 제거
df_fin = df_fin[['ItemID', 'Rating', 'Timestamp', 'review', '소재지(도로명)']]
df_fin.dropna(inplace=True)

# 결과 확인
df_fin.head()

,ItemID,Rating,Timestamp,review,소재지(도로명)
0,가무댕댕,3.5,2021.10.13,추억의 가무댕댕\n밥 양 좀 늘려주셨으면 좋겠어요. 서빙하는 남자분도 좀 불친절,울산광역시 남구 대학로94번길 11 (무거동)
1,가무댕댕,4.0,2021.10.09,"물려서 다 못먹은 돈까스 ,, 예전에 비해 맛이 많이 달라졌다",울산광역시 남구 대학로94번길 11 (무거동)
2,가무댕댕,4.5,2021.10.01,맛잇어요,울산광역시 남구 대학로94번길 11 (무거동)
3,가무댕댕,5.0,2021.09.29,맛있게 잘 먹었어요,울산광역시 남구 대학로94번길 11 (무거동)
4,가무댕댕,5.0,2021.09.25,맛있게 잘 먹었습니다~♡,울산광역시 남구 대학로94번길 11 (무거동)


In [ ]:
# 소재지 데이터를 split하여 따로 저장
df_fin['add_summary_1'] = df_fin['소재지(도로명)']
df_fin['add_summary_2'] = df_fin['소재지(도로명)']

for idx, i in enumerate(df_fin['add_summary_1']) :
    try :
        result = list(str(i).split())[1]
        df_fin['add_summary_1'][idx] = result
    except :
      pass

for idx, i in enumerate(df_fin['add_summary_2']) :
    try :
        result = list(str(i).split())[2]
        df_fin['add_summary_2'][idx] = result
    except :
      pass

# 결과 확인
df_fin = df_fin[['add_summary_1', 'add_summary_2', 'ItemID', 'Timestamp', 'Rating', 'review']]
df_fin.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,add_summary_1,add_summary_2,ItemID,Timestamp,Rating,review
0,남구,대학로94번길,가무댕댕,2021.10.13,3.5,추억의 가무댕댕\n밥 양 좀 늘려주셨으면 좋겠어요. 서빙하는 남자분도 좀 불친절
1,남구,대학로94번길,가무댕댕,2021.10.09,4.0,"물려서 다 못먹은 돈까스 ,, 예전에 비해 맛이 많이 달라졌다"
2,남구,대학로94번길,가무댕댕,2021.10.01,4.5,맛잇어요
3,남구,대학로94번길,가무댕댕,2021.09.29,5.0,맛있게 잘 먹었어요
4,남구,대학로94번길,가무댕댕,2021.09.25,5.0,맛있게 잘 먹었습니다~♡


## **3. 리뷰 데이터 전처리**

In [ ]:
# 리뷰 데이터 전처리
df_fin['review_pre'] = df_fin['review']
text_desc = []
for text in df_fin['review'] :
    text = text.replace('\n', ' ').strip()
    doc_tokens = text.replace('  ', ' ').replace('   ', ' ').replace('    ', ' ').replace('     ', ' ')
    doc_tokens = doc_tokens.lower()
    text_desc.append(doc_tokens)

df_fin.loc[:]['review_pre'] = text_desc

# 결과 확인
df_fin = df_fin[['add_summary_1', 'add_summary_2', 'ItemID', 'Timestamp', 'Rating', 'review_pre']]
df_fin.head()

,add_summary_1,add_summary_2,ItemID,Timestamp,Rating,review_pre
0,남구,대학로94번길,가무댕댕,2021.10.13,3.5,추억의 가무댕댕 밥 양 좀 늘려주셨으면 좋겠어요. 서빙하는 남자분도 좀 불친절
1,남구,대학로94번길,가무댕댕,2021.10.09,4.0,"물려서 다 못먹은 돈까스 ,, 예전에 비해 맛이 많이 달라졌다"
2,남구,대학로94번길,가무댕댕,2021.10.01,4.5,맛잇어요
3,남구,대학로94번길,가무댕댕,2021.09.29,5.0,맛있게 잘 먹었어요
4,남구,대학로94번길,가무댕댕,2021.09.25,5.0,맛있게 잘 먹었습니다~♡


In [ ]:
# export data
df_fin.to_csv('dir/', header=True, index=False)